In [12]:
import pandas as pd
import numpy as np

In [13]:
# 1. Create date range for 1 year at 4-hour intervals
dates = pd.date_range(start="2024-03-01", end="2025-3-21", freq="4h")
df = pd.DataFrame({"date": dates})

In [14]:
def generate_synthetic_data(row):
    month = row["date"].month
    hour = row["date"].hour
    
    # Season-specific base values
    if 6 <= month <= 8:    # Summer (Jun-Aug)
        base_temp = np.random.normal(35, 5)     # 35°C ±5
        base_humidity = np.random.normal(40, 10)  # 40% ±10
    elif month in [12, 1, 2]:  # Winter (Dec-Feb)
        base_temp = np.random.normal(20, 3)     # 20°C ±3
        base_humidity = np.random.normal(70, 5)  # 70% ±5
    else:  # Spring/Autumn (Mar-May, Sep-Nov)
        base_temp = np.random.normal(25, 4)     # 25°C ±4
        base_humidity = np.random.normal(55, 8)  # 55% ±8

    # Diurnal adjustments (season-dependent)
    if 8 <= hour < 12:  # Midday (8 AM - 12 PM)
        if 6 <= month <= 8:    # Summer: large temp rise, humidity drop
            base_temp += 7
            base_humidity -= 15
        elif month in [12, 1, 2]:  # Winter: smaller changes
            base_temp += 3
            base_humidity -= 5
        else:  # Spring/Autumn
            base_temp += 5
            base_humidity -= 10

    # Clip humidity to 0-100% and add noise
    temp = base_temp + np.random.normal(0, 2)
    humidity = np.clip(base_humidity + np.random.normal(0, 3), 0, 100)
    
    return temp, humidity

In [15]:
df[["temperature", "humidity"]] = df.apply(generate_synthetic_data, axis=1, result_type="expand")

In [16]:
def calculate_verdict(row):
    humidity = row["humidity"]
    temperature = row["temperature"]
    month = row["date"].month
    hour = row["date"].hour

    # 1. Dynamic threshold: 50% base, adjusted by temp (non-linear)
    base_threshold = 50
    if temperature > 25:  # Only reduce threshold above 25°C
        temp_adjusted_threshold = base_threshold - (temperature - 25) * 0.8
    else:
        temp_adjusted_threshold = base_threshold

    # 2. Seasonal adjustments (summer/winter only)
    if 6 <= month <= 8:    # Summer: lower threshold
        temp_adjusted_threshold -= 8
    elif month in [12, 1, 2]:  # Winter: raise threshold
        temp_adjusted_threshold += 10

    # 3. Time-of-day suppression (10 PM - 4 AM: no irrigation)
    if 22 <= hour or hour < 4:
        temp_adjusted_threshold = 100  # Disable irrigation

    # 4. Sigmoid probability with noise
    humidity_deficit = temp_adjusted_threshold - humidity
    prob_irrigate = 1 / (1 + np.exp(-0.15 * (humidity_deficit + np.random.normal(0, 2))))
    
    # 5. Final verdict (80% sigmoid, 20% randomness for edge cases)
    if np.random.rand() < 0.2:  # Random override
        return np.random.choice([0, 1], p=[0.7, 0.3])  # Bias towards no irrigation
    
    return 1 if prob_irrigate > 0.65 else 0

In [17]:
df["verdict"] = df.apply(calculate_verdict, axis=1)

In [18]:
df.to_csv("Better_Dataset.csv", index=False)